In [4]:
import pandas as  pd
import folium

In [2]:
data = pd.read_csv("../data/clean_torun_historical_monuments.csv")

In [3]:
data.head()

,Unnamed: 0,image.value,itemLabel.value,lat.value,lon.value
0,0,http://commons.wikimedia.org/wiki/Special:File...,Pomnik Mikołaja Kopernika w Toruniu,53.010278,18.605000
1,1,http://commons.wikimedia.org/wiki/Special:File...,Fort IV Twierdzy Toruń,53.040833,18.632778
2,2,http://commons.wikimedia.org/wiki/Special:File...,Dwór Artusa w Toruniu,53.009944,18.604472
3,3,http://commons.wikimedia.org/wiki/Special:File...,Fabryka Born & Schütze,53.024200,18.610400
4,4,http://commons.wikimedia.org/wiki/Special:File...,Brama Klasztorna w Toruniu,53.008277,18.603617


In [20]:
data['category'] = data['itemLabel.value'].apply(lambda x: 'Pomniki' if 'pomnik' in x.lower() else 'Budynki historyczne')

In [21]:
data.head()

,Unnamed: 0,image.value,itemLabel.value,lat.value,lon.value,category
0,0,http://commons.wikimedia.org/wiki/Special:File...,Pomnik Mikołaja Kopernika w Toruniu,53.010278,18.605000,Pomniki
1,1,http://commons.wikimedia.org/wiki/Special:File...,Fort IV Twierdzy Toruń,53.040833,18.632778,Budynki historyczne
2,2,http://commons.wikimedia.org/wiki/Special:File...,Dwór Artusa w Toruniu,53.009944,18.604472,Budynki historyczne
3,3,http://commons.wikimedia.org/wiki/Special:File...,Fabryka Born & Schütze,53.024200,18.610400,Budynki historyczne
4,4,http://commons.wikimedia.org/wiki/Special:File...,Brama Klasztorna w Toruniu,53.008277,18.603617,Budynki historyczne


In [13]:
torun_center = [53.0138, 18.5984]
m = folium.Map(location=torun_center, zoom_start=15, tiles="CartoDB positron")

In [22]:
#add folium layers

pomniki_layer = folium.FeatureGroup(name='Pomniki')
budynki_layer = folium.FeatureGroup(name='Budynki historyczne')

In [28]:
for _, row in data.iterrows():
    image_url = row['image.value']
    html = f"""
    <div style="background-color: rgba(255,255,255,0.8); padding: 10px; border-radius: 10px;">
        <h4>{row['itemLabel.value']}</h4>
        <img src="{image_url}" width="200">
        <p>Lat: {row['lat.value']}, Lon: {row['lon.value']}</p>
    </div>
    """
    iframe = folium.IFrame(html=html, width=220, height=250)
    popup = folium.Popup(iframe, max_width=250)

    marker = folium.Marker(
        location=[row['lat.value'], row['lon.value']],
        popup=popup,
        tooltip=row['itemLabel.value'],
        icon=folium.Icon(color='blue' if row['category']=='Pomniki' else 'green', icon='info-sign')
    )

    # add layer
    if row['category'] == 'Pomniki':
        pomniki_layer.add_child(marker)
    else:
        budynki_layer.add_child(marker)


In [29]:
#add layers to map
pomniki_layer.add_to(m)
budynki_layer.add_to(m)

# add mapcontrol
folium.LayerControl(collapsed=False).add_to(m)

In [31]:
m #show map in notbook

In [32]:
#save mapt html
m.save("torun_map_layered.html")